In [336]:
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import os
from pathlib import Path
import re
import pickle
# ver qtos o modelo apenas de ner acertaria
from transformers import AutoTokenizer, AutoModelForTokenClassification
import nltk    
from nltk import tokenize 
import torch
from transformers import BertTokenizer,BertForTokenClassification
import numpy as np
import json   
from importlib import reload  # Python 3.4+
import random
import model as mod
from model import BertForChunkClassification
from transformers import AdamW, BertConfig, get_linear_schedule_with_warmup

from torch.utils.data import DataLoader
from importlib import reload 
#from eval import predict
import eval
#import importlib
#importlib.reload(module)
import dataset
from dataset import InputFeatures, load_and_cache_examples
import functionsAval as f

In [44]:
f = reload(f)
reload(dataset)
reload(eval)
reload(mod)

# em numero de frases
BATCH=506
#BATCH=5
#BATCH=800
#BATCH=8000 
print('BATCH:', BATCH)

BATCH: 506


In [118]:
dicSentences_new_test = f.loadSentencesTest()
print(len(dicSentences_new_test))
dicSentences_new_test = {k: v for k, v in dicSentences_new_test.items() if k<BATCH}
print(dicSentences_new_test[0])
#print(dicSentences_new_test[27])
print('numero de sentencas no total:', len(dicSentences_new_test))

sentences=list()
for key, value in dicSentences_new_test.items():
    if key<BATCH:
        tokens = value[0]
        tokens = [tok[0] for tok in tokens]
        sentences.append(' '.join(tokens).strip())
#print(sentences[0])

tags, tokens = f.predictBERTNER_IO(sentences, 'all')
dic_predictions = f.getDicPredictions(tags, tokens)
print(dic_predictions[0])
print('len(dic_predictions):', len(dic_predictions))
#print(dic_predictions[9])
f.save_obj('dic_predictions_results_ner_'+str(BATCH), dic_predictions)
#dic_predictions = f.load_obj('dic_predictions_results_ner_'+str(BATCH))
print('verificando dados:')
#for key, value in dic_predictions.items():
#    print('key:',key)
#    print(dic_predictions[key])
#    if key>2:
#        break
        
print('len(dicSentences_new_test):', len(dicSentences_new_test))
print('len(dic_predictions):', len(dic_predictions))

region_true_list, region_pred_list, lista_erros = f.getListaRegionsTruePred(BATCH, dicSentences_new_test, dic_predictions)
f.save_obj('region_true_list'+str(BATCH), region_true_list)
print('region_pred_list[:4]:', region_pred_list[:4])
print('region_true_list[:4]:', region_true_list[:4])
print('lista_erros[:8]:', lista_erros[:8])
print('len(lista_erros):', len(lista_erros))
print('len(set(lista_erros)):', len(set(lista_erros)))
#print(dic_predictions[8])
#print(dicSentences_new_test[8][1])
print('len(region_true_list):', len(region_true_list))
print('len(region_pred_list):', len(region_pred_list))
#print('pred:',region_pred_list[:15])
#print('true:',region_true_list[:15])

print('-----Avaliando só modelo de NER:-----')

print(classification_report(region_true_list, region_pred_list, digits=6))
print(confusion_matrix(region_true_list, region_pred_list))


Pegando sentencas de teste gabarito: dic_sentencesTest.pkl
506
[[['Lucas', 0, 43], [',', 1, 48], ['74', 2, 50], ['anos', 3, 53], ['.', 4, 57]], []]
numero de sentencas no total: 506
idx2tag: {0: 'Teste', 1: 'Anatomia', 2: 'O', 3: 'Problema', 4: 'Tratamento', 5: '<pad>'}
[[['Lucas', 0], [',', 1], ['74', 2], ['anos', 3], ['.', 4]], []]
len(dic_predictions): 506
verificando dados:
len(dicSentences_new_test): 506
len(dic_predictions): 506
region_pred_list[:4]: ['Problema', 'Tratamento', 'Problema', 'Problema']
region_true_list[:4]: ['Problema', 'Tratamento', 'Problema', 'Problema']
lista_erros[:8]: [7, 8, 13, 13, 14, 15, 15, 15]
len(lista_erros): 383
len(set(lista_erros)): 169
len(region_true_list): 1140
len(region_pred_list): 1140
-----Avaliando só modelo de NER:-----
              precision    recall  f1-score   support

    Anatomia   0.829268  0.346939  0.489209       196
           O   0.000000  0.000000  0.000000       149
    Problema   0.798319  0.843195  0.820144       338
       

In [46]:
tags[0]

['O', 'O', 'O', 'O', 'O']

In [47]:
tokens[0]

['Lucas', ',', '74', 'anos', '.']

## CRF primeiro nivel

In [48]:
dicPostagger = f.load_obj('dic_postagger')
def tipoPostaggerTokens(token, dicPostagger):
    postagger = 'N' # na duvida é N
    if token.lower() in dicPostagger.keys():
        postagger = dicPostagger.get(token.lower())
    return postagger
tipoPostaggerTokens('coração', dicPostagger)

'N'

In [49]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import re
def has_numbers(inputString):
    return str(bool(re.search(r'\d', inputString)))

print(has_numbers('metionina 5mg'))

def isstopword(p):
  return str(p in stopwords.words('portuguese'))

print(isstopword('pelo'))


True
True


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lisat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [309]:
# binario

CLUSTER=300

def getTiposEntidade():
    return ['O','ENT']

def read_clusters(cluster_file):
    word2cluster = {}
    try:
        with open(cluster_file, encoding='utf-8') as i:
            for num, line in enumerate(i):
                if line:
                    word, cluster = line.strip().split('\t')
                    word2cluster[word] = cluster
    except:
        raise
    return word2cluster

def word2features(sent, i):
  #if str(sent[i][0])!='nan':#esto es extra
    word = sent[i][0]
    postag = tipoPostaggerTokens(word, dicPostagger)
    cluster = word2cluster[word.lower()] if word.lower() in word2cluster else "0"
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[:3]': word[:3],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'word.cluster': cluster
    }
    if i > 0:
        word1 = sent[i-1][0]
        #postag1 = sent[i-1][1]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:word.cluster': cluster
        })
    else:
        features['BOS'] = True
    
    if i > 1 and (JANELA==2 or JANELA==3 or JANELA==4):
        word1 = sent[i-2][0]
        #postag1 = sent[i-2][1]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-2:word.lower()': word1.lower(),
            '-2:word.istitle()': word1.istitle(),
            '-2:word.isupper()': word1.isupper(),
            '-2:postag': postag1,
            '-2:word.cluster': cluster
        })
    else:
        features['Second_word'] = True

    if i > 2 and (JANELA==3 or JANELA==4):
        word1 = sent[i-1][0]
        #postag1 = sent[i-1][1]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-3:word.lower()': word1.lower(),
            '-3:word.istitle()': word1.istitle(),
            '-3:word.isupper()': word1.isupper(),
            '-3:postag': postag1,
            '-3:word.cluster': cluster
        })
    else:
        features['Third_word'] = True
    
    if i > 3 and JANELA==4:
        word1 = sent[i-2][0]
        #postag1 = sent[i-2][1]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-4:word.lower()': word1.lower(),
            '-4:word.istitle()': word1.istitle(),
            '-4:word.isupper()': word1.isupper(),
            '-4:postag': postag1,
            '-4:word.cluster': cluster
        })
    else:
        features['Fourth_word'] = True
    
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        #postag1 = sent[i+1][1]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:word.cluster': cluster
        })
    else:
        features['EOS'] = True
    if i < len(sent)-2 and (JANELA==2 or JANELA==3 or JANELA==4):
        word1 = sent[i+2][0]
        #postag1 = sent[i+2][1]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+2:word.lower()': word1.lower(),
            '+2:word.istitle()': word1.istitle(),
            '+2:word.isupper()': word1.isupper(),
            '+2:postag': postag1,
            '+2:word.cluster': cluster
        })
    else:
        features['Second_to_last'] = True
    if i < len(sent)-3 and (JANELA==3 or JANELA==4):
        word1 = sent[i+3][0]
        #postag1 = sent[i+3][1]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+3:word.lower()': word1.lower(),
            '+3:word.istitle()': word1.istitle(),
            '+3:word.isupper()': word1.isupper(),
            '+3:postag': postag1,
            '+3:word.cluster': cluster
        })
    else:
        features['Third_to_last'] = True
    
    if i < len(sent)-4 and JANELA==4:
        word1 = sent[i+4][0]
        #postag1 = sent[i+4][1]
        postag1 = tipoPostaggerTokens(word1, dicPostagger)
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+4:word.lower()': word1.lower(),
            '+4:word.istitle()': word1.istitle(),
            '+4:word.isupper()': word1.isupper(),
            '+4:postag': postag1,
            '+4:word.cluster': cluster
        })
    else:
        features['Fourth_to_last'] = True
    

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

# gen = (x for x in xyz if x not in a)
def sent2labels(sent):
    retorno = list()
    for token, label in sent:
      if label =='O':
        retorno.append(label)
      else:
        retorno.append('ENT')
    return retorno
    #return ['ENT' if label!='O' elif 'O' for token, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

word2cluster = read_clusters(r"cluster/cluster-"+str(CLUSTER)+".tsv")


In [310]:
pathTest=r'C:\Users\lisat\OneDrive\jupyter notebook\spanclassification\preProcessamento\data-ner/nested_test.conll'

with open(pathTest, encoding='utf-8') as fi:
  testdata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in fi.read().split('\n\n')]

X_test = [sent2features(s) for s in testdata]
y_test = [sent2labels(s) for s in testdata]

In [311]:
X_test[0]

[{'bias': 1.0,
  'word.lower()': 'lucas',
  'word[-3:]': 'cas',
  'word[:3]': 'Luc',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  'postag': 'N',
  'word.cluster': '0',
  'BOS': True,
  'Second_word': True,
  'Third_word': True,
  'Fourth_word': True,
  '+1:word.lower()': ',',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:postag': 'PU',
  '+1:word.cluster': '22',
  '+2:word.lower()': '74',
  '+2:word.istitle()': False,
  '+2:word.isupper()': False,
  '+2:postag': 'NUM',
  '+2:word.cluster': '299',
  '+3:word.lower()': 'anos',
  '+3:word.istitle()': False,
  '+3:word.isupper()': False,
  '+3:postag': 'N',
  '+3:word.cluster': '134',
  'Fourth_to_last': True},
 {'bias': 1.0,
  'word.lower()': ',',
  'word[-3:]': ',',
  'word[:3]': ',',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  'postag': 'PU',
  'word.cluster': '22',
  '-1:word.lower()': 'lucas',
  '-1:word.istitle()': True,
  '-1:word.isupper

In [312]:
BATCH

800

In [313]:
import joblib
import os

#AVAL no conjunto de teste do ner (flat)

#def getTiposEntidade():
    #return ['Problema','Teste','Tratamento','Anatomia']

print('Binario flat, clustr 300, janela 3')
OUTPUT_PATH = "CRF"
OUTPUT_FILE = "crf_model_primeiro_nivel_binario"
crf = joblib.load(os.path.join(OUTPUT_PATH, OUTPUT_FILE))
y_pred = crf.predict(X_test)

from sklearn_crfsuite.metrics import flat_f1_score, flat_classification_report

finalScore = flat_f1_score(y_test, y_pred, average='weighted', labels=getTiposEntidade())
print("F1 weighted:",finalScore)

print(flat_classification_report(
    y_test, y_pred, labels=getTiposEntidade(), digits=3
))



Binario flat, clustr 300, janela 3
F1 weighted: 0.9404560516998399
              precision    recall  f1-score   support

           O      0.959     0.956     0.957      3809
         ENT      0.898     0.905     0.901      1644

    accuracy                          0.940      5453
   macro avg      0.929     0.930     0.929      5453
weighted avg      0.941     0.940     0.940      5453



In [72]:
def read_clusters(cluster_file):
    word2cluster = {}
    try:
        with open(cluster_file, encoding='utf-8') as i:
            for num, line in enumerate(i):
                if line:
                    word, cluster = line.strip().split('\t')
                    word2cluster[word] = cluster
    except:
        raise
    return word2cluster

def word2features(sent, i):
  #if str(sent[i][0])!='nan':#esto es extra
    word = sent[i][0]
    postag = tipoPostaggerTokens(word, dicPostagger)
    cluster = word2cluster[word.lower()] if word.lower() in word2cluster else "0"
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[:3]': word[:3],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'word.cluster': cluster
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:word.cluster': cluster
        })
    else:
        features['BOS'] = True
    
    if i > 1 and (JANELA==2 or JANELA==3 or JANELA==4):
        word1 = sent[i-2][0]
        postag1 = sent[i-2][1]
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-2:word.lower()': word1.lower(),
            '-2:word.istitle()': word1.istitle(),
            '-2:word.isupper()': word1.isupper(),
            '-2:postag': postag1,
            '-2:word.cluster': cluster
        })
    else:
        features['Second_word'] = True

    if i > 2 and (JANELA==3 or JANELA==4):
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-3:word.lower()': word1.lower(),
            '-3:word.istitle()': word1.istitle(),
            '-3:word.isupper()': word1.isupper(),
            '-3:postag': postag1,
            '-3:word.cluster': cluster
        })
    else:
        features['Third_word'] = True
    
    if i > 3 and JANELA==4:
        word1 = sent[i-2][0]
        postag1 = sent[i-2][1]
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '-4:word.lower()': word1.lower(),
            '-4:word.istitle()': word1.istitle(),
            '-4:word.isupper()': word1.isupper(),
            '-4:postag': postag1,
            '-4:word.cluster': cluster
        })
    else:
        features['Fourth_word'] = True
    
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:word.cluster': cluster
        })
    else:
        features['EOS'] = True
    if i < len(sent)-2 and (JANELA==2 or JANELA==3 or JANELA==4):
        word1 = sent[i+2][0]
        postag1 = sent[i+2][1]
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+2:word.lower()': word1.lower(),
            '+2:word.istitle()': word1.istitle(),
            '+2:word.isupper()': word1.isupper(),
            '+2:postag': postag1,
            '+2:word.cluster': cluster
        })
    else:
        features['Second_to_last'] = True
    if i < len(sent)-3 and (JANELA==3 or JANELA==4):
        word1 = sent[i+3][0]
        postag1 = sent[i+3][1]
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+3:word.lower()': word1.lower(),
            '+3:word.istitle()': word1.istitle(),
            '+3:word.isupper()': word1.isupper(),
            '+3:postag': postag1,
            '+3:word.cluster': cluster
        })
    else:
        features['Third_to_last'] = True
    
    if i < len(sent)-4 and JANELA==4:
        word1 = sent[i+4][0]
        postag1 = sent[i+4][1]
        cluster = word2cluster[word1.lower()] if word1.lower() in word2cluster else "0"
        features.update({
            '+4:word.lower()': word1.lower(),
            '+4:word.istitle()': word1.istitle(),
            '+4:word.isupper()': word1.isupper(),
            '+4:postag': postag1,
            '+4:word.cluster': cluster
        })
    else:
        features['Fourth_to_last'] = True
    

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

#word2cluster = read_clusters(r"cluster/cluster-"+str(CLUSTER)+".tsv")


In [73]:
testdata[18]

[('Abdomen', 'Anatomia'),
 ('flácido', 'O'),
 (',', 'O'),
 ('indolor', 'O'),
 (',', 'O'),
 ('sem', 'O'),
 ('visceromegalias', 'Problema'),
 ('.', 'O')]

In [74]:
y_test[1]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'ENT',
 'O',
 'O',
 'O',
 'ENT',
 'ENT',
 'O',
 'O',
 'O',
 'O',
 'O']

In [75]:
X_test[1]

[{'bias': 1.0,
  'word.lower()': 'em',
  'word[-3:]': 'Em',
  'word[:3]': 'Em',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  'postag': 'PREP',
  'word.cluster': '164',
  'BOS': True,
  'Second_word': True,
  'Third_word': True,
  'Fourth_word': True,
  '+1:word.lower()': 'acompanhamento',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:postag': 'N',
  '+1:word.cluster': '34',
  '+2:word.lower()': 'no',
  '+2:word.istitle()': False,
  '+2:word.isupper()': False,
  '+2:postag': 'ART',
  '+2:word.cluster': '164',
  '+3:word.lower()': 'ambualtorio',
  '+3:word.istitle()': False,
  '+3:word.isupper()': False,
  '+3:postag': 'N',
  '+3:word.cluster': '0',
  'Fourth_to_last': True},
 {'bias': 1.0,
  'word.lower()': 'acompanhamento',
  'word[-3:]': 'nto',
  'word[:3]': 'aco',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  'postag': 'N',
  'word.cluster': '34',
  '-1:word.lower()': 'em',
  '-1:word.istitl

In [83]:
import joblib
import os

#AVAL no conjunto de teste do ner (flat)
print('NER flat, clustr 50, janela 3')


def getTiposEntidade():
    return ['Problema','Teste','Tratamento','Anatomia']

JANELA=3
CLUSTER=50

word2cluster = read_clusters(r"cluster/cluster-"+str(CLUSTER)+".tsv")
X_test = [sent2features(s) for s in testdata]
y_test = [sent2labels(s) for s in testdata]

OUTPUT_PATH = "CRF"
OUTPUT_FILE = "crf_model_primeiro_nivel"
crf = joblib.load(os.path.join(OUTPUT_PATH, OUTPUT_FILE))
y_pred = crf.predict(X_test)

from sklearn_crfsuite.metrics import flat_f1_score, flat_classification_report

finalScore = flat_f1_score(y_test, y_pred, average='weighted', labels=getTiposEntidade())
print("F1 weighted:",finalScore)

print(flat_classification_report(
    y_test, y_pred, labels=getTiposEntidade(), digits=3
))



NER flat, clustr 50, janela 3
F1 weighted: 0.9378300780433376
              precision    recall  f1-score   support

    Problema      0.948     0.961     0.954       795
       Teste      0.935     0.893     0.914       308
  Tratamento      0.968     0.939     0.953       446
    Anatomia      0.919     0.716     0.805        95

   micro avg      0.950     0.928     0.939      1644
   macro avg      0.942     0.877     0.907      1644
weighted avg      0.949     0.928     0.938      1644



In [314]:
dicSentences_new_test = f.loadSentencesTest()

BATCH=800
sentences=list()
frases=list()
allTokens=list()
listatokens=list()
for key, value in dicSentences_new_test.items():
    if key<BATCH:
        tokens = value[0]
        for token in tokens:
            listatokens.append(token[0])
            if len(token)>0:
                frases.append(tuple(token))
        sentences.append(frases)
        allTokens.append(listatokens)
        listatokens=list()
        frases=list()
print('sentences[0]:', sentences[0])
print(allTokens[0])

Pegando sentencas de teste gabarito: dic_sentencesTest.pkl
sentences[0]: [('Lucas', 0, 43), (',', 1, 48), ('74', 2, 50), ('anos', 3, 53), ('.', 4, 57)]
['Lucas', ',', '74', 'anos', '.']


In [87]:
# BATCH 800

dicSentences_new_test = f.loadSentencesTest()
print(len(dicSentences_new_test))
dicSentences_new_test = {k: v for k, v in dicSentences_new_test.items() if k<BATCH}
print(dicSentences_new_test[0])
#print(dicSentences_new_test[27])
print('numero de sentencas no total:', len(dicSentences_new_test))

'''
example_sent = testdata[18]
print("\nSentence:", ' '.join(sent2tokens(example_sent)))
print(' '.join(sent2tokens(example_sent)))
print("Predicted:", ' '.join(crf.predict([sent2features(example_sent)])[0]))
print("Correct:  ", ' '.join(sent2labels(example_sent)))
'''

sentences=list()
frases=list()
allTokens=list()
tokens=list()
sentences=list()
frases=list()
allTokens=list()
#listatokens=list()
for key, value in dicSentences_new_test.items():
    if key<BATCH:
        tokens = value[0]
        for token in tokens:
            listatokens.append(token[0])
            if len(token)>0:
                frases.append(tuple(token))
        sentences.append(frases)
        allTokens.append(listatokens)
        listatokens=list()
        frases=list()
print('sentences[0]:', sentences[0])
print(allTokens[0])

#testdata = [[tuple(w.split(' ')) for w in snt.split('\n')] for snt in f.read().split('\n\n')]

def getTiposEntidade():
    return ['Problema','Teste','Tratamento','Anatomia']

#X_test = [sent2tokensPredict(s) for s in sentences]
X_test = [sent2features(s) for s in sentences]
#print('X_test[0]:', X_test[0])
print('len(sentences):', len(sentences))
print('len(allTokens):', len(allTokens))

OUTPUT_PATH = "CRF"
OUTPUT_FILE = "crf_model_primeiro_nivel"
crf = joblib.load(os.path.join(OUTPUT_PATH, OUTPUT_FILE))
y_pred = crf.predict(X_test)
print('len(y_pred):', len(y_pred))
print('y_pred[1]:', y_pred[1])
print('allTokens[1]:', allTokens[1])

#for tags, tokens in zip(y_pred, allTokens):
dic_predictions = f.getDicPredictions(y_pred, allTokens)
region_true_list, region_pred_list, lista_erros = f.getListaRegionsTruePred(BATCH, dicSentences_new_test, dic_predictions)
print('-----Avaliando só modelo de NER com CRF, nested:-----')
print(classification_report(region_true_list, region_pred_list, digits=6))
print(confusion_matrix(region_true_list, region_pred_list))

#tags, tokens = f.predictBERTNER_IO(sentences, 'all')
#dic_predictions = f.getDicPredictions(tags, tokens)


Pegando sentencas de teste gabarito: dic_sentencesTest.pkl
506
[[['Lucas', 0, 43], [',', 1, 48], ['74', 2, 50], ['anos', 3, 53], ['.', 4, 57]], []]
numero de sentencas no total: 506
sentences[0]: [('Lucas', 0, 43), (',', 1, 48), ('74', 2, 50), ('anos', 3, 53), ('.', 4, 57)]
['Lucas', ',', '74', 'anos', '.']
len(sentences): 506
len(allTokens): 506
len(y_pred): 506
y_pred[1]: ['O', 'O', 'O', 'Problema', 'O', 'O', 'O', 'O', 'Problema', 'O', 'Tratamento', 'O', 'Tratamento', 'Tratamento', 'O', 'O', 'O', 'O', 'O']
allTokens[1]: ['Em', 'acompanhamento', 'no', 'ambualtorio', 'há', '5', 'anos', 'por', 'FA', ',', 'uso', 'de', 'marevan', '5mg', '1', 'x', 'ao', 'dia', '.']
-----Avaliando só modelo de NER com CRF, nested:-----
              precision    recall  f1-score   support

    Anatomia   0.669725  0.372449  0.478689       196
           O   0.000000  0.000000  0.000000       605
    Problema   0.362044  0.733728  0.484848       338
       Teste   0.701068  0.810700  0.751908       243
  Tra

### AVAL spans / regioes

In [88]:
dic_predictions[1]

[[['Em', 0],
  ['acompanhamento', 1],
  ['no', 2],
  ['ambualtorio', 3],
  ['há', 4],
  ['5', 5],
  ['anos', 6],
  ['por', 7],
  ['FA', 8],
  [',', 9],
  ['uso', 10],
  ['de', 11],
  ['marevan', 12],
  ['5mg', 13],
  ['1', 14],
  ['x', 15],
  ['ao', 16],
  ['dia', 17],
  ['.', 18]],
 [['ambualtorio', [3], 'Problema'],
  ['FA', [8], 'Problema'],
  ['uso', [10], 'Tratamento'],
  ['marevan 5mg', [12, 13], 'Tratamento']]]

In [96]:
NUM_JANELA=4
#def getListaPositiva(dic_sentences):
#dic_sentences = dic_sentencesTest
def getListaCombinacoes(dic_sentences):
    listaSentencas=list()
    lista=list()
    numJanela=NUM_JANELA
    for key, value in dic_sentences.items():
        #print('value:', value)
        entidades = value[1]
        if len(entidades)==0:
            continue
        listaIndicesEntidadesFrase=list()
        for entidade in entidades:
          listaIndicesEntidadesFrase.append(entidade[1])
        dicTokens={}
        for token in value[0]:
          #print('token:', token)
          dicTokens[token[1]]=token[0]
        for entidade in entidades:
            #print('entidade:', entidade)
            label = entidade[2]
            indiceEntidade1=entidade[1][0]
            indiceEntidade2=entidade[1][-1]
            vizinhosAntes = list()
            vizinhosDepois = list()
            #print('indiceEntidade:', indiceEntidade)
            for tokens in value[0]:
                indice=tokens[1]
                #print('token: {}, indice: {}'.format(tokens[0], indice))
                if indice+4>=indiceEntidade1 and indice<indiceEntidade1:
                    vizinhosAntes.append(tokens[0])
                if indice-4<=indiceEntidade2 and indice>indiceEntidade2:
                    vizinhosDepois.append(tokens[0])
            lista.append([entidade[0], vizinhosAntes, vizinhosDepois, label])
            # agora, os negativos, para entidades com mais de um token
            if len(entidade[0].split())>1:
              #print('entidade[0] mais de um token: {}'.format(entidade[0].split()))
              #listaCombinacoes = powerset(entidade[0].split())
              listaCombinacoes = powerset(entidade[1])
              #print(listaCombinacoes)
              # ver se alguma dessas é positiva
              for combinacao in listaCombinacoes:
                #print('combinacao:', combinacao)
                if combinacao in listaIndicesEntidadesFrase:
                  #print('teeeem:', combinacao)
                  pass
                else:
                  combinacaoTokens = [dicTokens[t] for t in combinacao]
                  vizinhosAntes = list()
                  vizinhosDepois = list()
                  indiceEntidade1=combinacao[0]
                  indiceEntidade2=combinacao[-1]
                  for tokens in value[0]:
                    indice=tokens[1]
                    #print('token: {}, indice: {}'.format(tokens[0], indice))
                    if indice+4>=indiceEntidade1 and indice<indiceEntidade1:
                        vizinhosAntes.append(tokens[0])
                    if indice-4<=indiceEntidade2 and indice>indiceEntidade2:
                        vizinhosDepois.append(tokens[0])
                  lista.append([' '.join(combinacaoTokens), vizinhosAntes, vizinhosDepois, 'O'])
        listaSentencas.append(lista)
        lista=list()
        #break
    return listaSentencas
listacombinacoes = getListaCombinacoes(dic_sentencesTest)
print(listacombinacoes[2])

[['HAS', [], ['há', '15', 'anos', 'em'], 'Problema'], ['losartana 50mg', ['anos', 'em', 'uso', 'de'], ['/', 'dia', 'e', 'digoxina'], 'Tratamento'], ['losartana', ['anos', 'em', 'uso', 'de'], ['50mg', '/', 'dia', 'e'], 'O'], ['50mg', ['em', 'uso', 'de', 'losartana'], ['/', 'dia', 'e', 'digoxina'], 'O'], ['digoxina', ['50mg', '/', 'dia', 'e'], ['1', '/', '2', 'cp'], 'Tratamento'], ['carvedilol 25', ['cp', '/', 'dia', ','], ['12', '/', '12', ','], 'Tratamento'], ['carvedilol', ['cp', '/', 'dia', ','], ['25', '12', '/', '12'], 'O'], ['25', ['/', 'dia', ',', 'carvedilol'], ['12', '/', '12', ','], 'O'], ['HCTZ', ['12', '/', '12', ','], ['.'], 'Tratamento']]


In [90]:
def powerset(entidade):    
    lista=list()
    for i in range(1, len(entidade)+1):
        #lista.append(entidade[i-1])
        for j in range(len(entidade) - i + 1):
            lista.append(entidade[j:j + i])
    return lista
        
#print(list(powerset([4, 5, 6])))
print(powerset(['dor', 'no', 'peito']))

[['dor'], ['no'], ['peito'], ['dor', 'no'], ['no', 'peito'], ['dor', 'no', 'peito']]


In [245]:
dic_sentencesTest = f.load_obj(r'dic_sentencesTest')
dic_sentencesTest[2]

[[['Comorbidades', 0, 142],
  [':', 1, 154],
  ['DM', 2, 156],
  ['há', 3, 159],
  ['10', 4, 162],
  ['anos', 5, 165],
  ['em', 6, 170],
  ['uso', 7, 173],
  ['de', 8, 177],
  ['metformina', 9, 180],
  ['850mg', 10, 191],
  ['3', 11, 197],
  ['cp', 12, 199],
  ['/', 13, 201],
  ['dia', 14, 202],
  [',', 15, 205],
  ['acarbose', 16, 207],
  ['1', 17, 216],
  ['cp', 18, 218],
  ['/', 19, 220],
  ['dia', 20, 221],
  ['e', 21, 225],
  ['glicazida', 22, 227],
  ['60mg', 23, 237],
  ['2', 24, 242],
  ['cp', 25, 244],
  ['/', 26, 246],
  ['dia', 27, 247],
  ['e', 28, 251],
  ['insulina', 29, 253],
  ['(', 30, 262],
  ['24', 31, 263],
  ['-', 32, 266],
  ['0', 33, 268],
  ['-', 34, 270],
  ['24', 35, 272],
  [')', 36, 274],
  ['.', 37, 275]],
 [['Comorbidades', [0], 'Problema'],
  ['DM', [2], 'Problema'],
  ['metformina 850mg', [9, 10], 'Tratamento'],
  ['acarbose', [16], 'Tratamento'],
  ['glicazida 60mg', [22, 23], 'Tratamento'],
  ['insulina', [29], 'Tratamento']]]

In [113]:
dic_predictions[2]

[[['Comorbidades', 0],
  [':', 1],
  ['DM', 2],
  ['há', 3],
  ['10', 4],
  ['anos', 5],
  ['em', 6],
  ['uso', 7],
  ['de', 8],
  ['metformina', 9],
  ['850mg', 10],
  ['3', 11],
  ['cp', 12],
  ['/', 13],
  ['dia', 14],
  [',', 15],
  ['acarbose', 16],
  ['1', 17],
  ['cp', 18],
  ['/', 19],
  ['dia', 20],
  ['e', 21],
  ['glicazida', 22],
  ['60mg', 23],
  ['2', 24],
  ['cp', 25],
  ['/', 26],
  ['dia', 27],
  ['e', 28],
  ['insulina', 29],
  ['(', 30],
  ['24', 31],
  ['-', 32],
  ['0', 33],
  ['-', 34],
  ['24', 35],
  [')', 36],
  ['.', 37]],
 [['Comorbidades', [0], 'Problema'],
  ['DM', [2], 'Problema'],
  ['metformina 850mg', [9, 10], 'Tratamento'],
  ['acarbose', [16], 'Tratamento'],
  ['glicazida 60mg', [22, 23], 'Tratamento'],
  ['insulina', [29], 'Tratamento']]]

In [108]:
#listacombinacoesTest = getListaCombinacoes(dic_sentencesTest)
listaTest[0:2]

[[['Lucas', [], [',', '74', 'anos', '.'], 0],
  [',', ['Lucas'], ['74', 'anos', '.'], 0],
  ['74', ['Lucas', ','], ['anos', '.'], 0],
  ['anos', ['Lucas', ',', '74'], ['.'], 0],
  ['.', [',', '74', 'anos'], [], 0],
  ['Lucas ,', [], ['74', 'anos', '.'], 0],
  [', 74', ['Lucas'], ['anos', '.'], 0],
  ['74 anos', ['Lucas', ','], ['.'], 0],
  ['anos .', ['Lucas', ',', '74'], [], 0],
  ['Lucas , 74', [], ['anos', '.'], 0],
  [', 74 anos', ['Lucas'], ['.'], 0],
  ['74 anos .', ['Lucas', ','], [], 0],
  ['Lucas , 74 anos', [], ['.'], 0],
  [', 74 anos .', ['Lucas'], [], 0],
  ['Lucas , 74 anos .', [], [], 0]],
 [['Em', [], ['acompanhamento', 'no', 'ambualtorio', 'há'], 0],
  ['acompanhamento', ['Em'], ['no', 'ambualtorio', 'há', '5'], 0],
  ['no', ['Em', 'acompanhamento'], ['ambualtorio', 'há', '5', 'anos'], 0],
  ['ambualtorio',
   ['Em', 'acompanhamento', 'no'],
   ['há', '5', 'anos', 'por'],
   0],
  ['há',
   ['acompanhamento', 'no', 'ambualtorio'],
   ['5', 'anos', 'por', 'FA'],
   0],


In [296]:
#def getListaPositiva(dic_sentences):
#dic_sentences = dic_sentencesTest
def getListaCombinacoes(dic_sentences):
    listaSentencas=list()
    lista=list()
    dicPred={}
    #numJanela=NUM_JANELA
    numJanela=4
    num1=-1
    num2=0
    for key, value in dic_sentences.items():
        num1=num1+1
        #print('value:', value)
        entidades = value[1]
        if len(entidades)==0:
            dicPred[num2]=num1
            continue
        dicPred[num2]=num1
        num2=num2+1
        listaIndicesEntidadesFrase=list()
        for entidade in entidades:
          listaIndicesEntidadesFrase.append(entidade[1])
        dicTokens={}
        for token in value[0]:
          #print('token:', token)
          dicTokens[token[1]]=token[0]
        for entidade in entidades:
            #print('entidade:', entidade)
            #label = entidade[2]
            label = 'ENT'
            indiceEntidade1=entidade[1][0]
            indiceEntidade2=entidade[1][-1]
            vizinhosAntes = list()
            vizinhosDepois = list()
            #print('indiceEntidade:', indiceEntidade)
            for tokens in value[0]:
                indice=tokens[1]
                #print('token: {}, indice: {}'.format(tokens[0], indice))
                if indice+numJanela>=indiceEntidade1 and indice<indiceEntidade1:
                    vizinhosAntes.append(tokens[0])
                if indice-numJanela<=indiceEntidade2 and indice>indiceEntidade2:
                    vizinhosDepois.append(tokens[0])
            lista.append([entidade[0], vizinhosAntes, vizinhosDepois, label])
            # agora, os negativos, para entidades com mais de um token
            if len(entidade[0].split())>1:
              #print('entidade[0] mais de um token: {}'.format(entidade[0].split()))
              #listaCombinacoes = powerset(entidade[0].split())
              listaCombinacoes = powerset(entidade[1])
              #print(listaCombinacoes)
              # ver se alguma dessas é positiva
              for combinacao in listaCombinacoes:
                #print('combinacao:', combinacao)
                if combinacao in listaIndicesEntidadesFrase:
                  #print('teeeem:', combinacao)
                  pass
                else:
                  combinacaoTokens = [dicTokens[t] for t in combinacao]
                  vizinhosAntes = list()
                  vizinhosDepois = list()
                  indiceEntidade1=combinacao[0]
                  indiceEntidade2=combinacao[-1]
                  for tokens in value[0]:
                    indice=tokens[1]
                    #print('token: {}, indice: {}'.format(tokens[0], indice))
                    if indice+numJanela>=indiceEntidade1 and indice<indiceEntidade1:
                        vizinhosAntes.append(tokens[0])
                    if indice-numJanela<=indiceEntidade2 and indice>indiceEntidade2:
                        vizinhosDepois.append(tokens[0])
                  lista.append([' '.join(combinacaoTokens), vizinhosAntes, vizinhosDepois, 'O'])
        listaSentencas.append(lista)
        lista=list()
        #break
    return listaSentencas, dicPred

listacombinacoesTest = getListaCombinacoes(dic_sentencesTest)


In [316]:
def getTiposEntidade():
    #return ['Problema','Teste','Tratamento','Anatomia']
    return ['O','ENT']

def read_clusters(cluster_file):
    word2cluster = {}
    try:
        with open(cluster_file, encoding='utf-8') as i:
            for num, line in enumerate(i):
                if line:
                    word, cluster = line.strip().split('\t')
                    word2cluster[word] = cluster
    except:
        print(line)
        print(num)
        raise
    return word2cluster


def word2features(sent, word2cluster, dicPostagger):
    try:
        features = list()
        entidades = sent[0]
        for entidade in entidades.split():
            postag = tipoPostaggerTokens(entidade, dicPostagger)
            features.extend([
            'bias',
            'word.lower=' + entidade.lower(),
            'word[-3:]=' + entidade[-3:],
            'word[:3]=' + entidade[:3],
            'word.isupper=%s' % entidade.isupper(),
            'word.istitle=%s' % entidade.istitle(),
            'word.isdigit=%s' % entidade.isdigit(),
            'word.cluster=%s' % word2cluster[entidade.lower()] if entidade.lower() in word2cluster else "0",
            'postag=' + postag
            ])
        # palavras anteriores
        vizinhosAntes = sent[1] 
        if len(vizinhosAntes)>0:
            for num, vizinhoAntes in enumerate(vizinhosAntes):
                word1 = vizinhoAntes
                postag1 =  tipoPostaggerTokens(vizinhoAntes, dicPostagger)
                features.extend([
                    '-'+str(num+1)+':word.lower=' + word1.lower(),
                    '-'+str(num+1)+':word.istitle=%s' % word1.istitle(),
                    '-'+str(num+1)+':word.isupper=%s' % word1.isupper(),
                    '-'+str(num+1)+':postag=' + postag1
                ])
        else:
            features.append('BOS')

        # próximas palavras
        vizinhosDepois = sent[2]
        if len(vizinhosDepois)>0:
            for num, vizinhoDepois in enumerate(vizinhosDepois):
                word1 = vizinhoDepois
                postag1 =  tipoPostaggerTokens(vizinhoDepois, dicPostagger)
                features.extend([
                    '+'+str(num+1)+':word.lower=' + word1.lower(),
                    '+'+str(num+1)+':word.istitle=%s' % word1.istitle(),
                    '+'+str(num+1)+':word.isupper=%s' % word1.isupper(),
                    '+'+str(num+1)+':postag=' + postag1
                ])
        else:
            features.append('EOS')
    except:
        print('sent:', sent)
        raise
    return features


def sent2features(lista, word2cluster, dicPostagger):
    #return word2features(lista, word2cluster, dicPostagger)
    return [word2features(lista[i], word2cluster, dicPostagger) for i in range(len(lista))]

def sent2labels(lista):
    #return [label for _, _, _, label in lista]
    return [label for _, _, _, label in lista]

#def sent2tokens(sent):
#    return [token for token, postag, label in sent]

#word2cluster = read_clusters(r"cluster/cluster-50.tsv")
#word2cluster = read_clusters(r"clusters/cluster-5.tsv")
#word2cluster = read_clusters(r"clusters/cluster-10.tsv")
#word2cluster = read_clusters(r"clusters/cluster-100.tsv")
#word2cluster = read_clusters(r"clusters/cluster-300.tsv")

In [115]:
X_test = [sent2features(s, word2cluster, dicPostagger) for s in listacombinacoesTest]
y_test = [sent2labels(s) for s in listacombinacoesTest]

In [112]:
CLUSTER=300
word2cluster = read_clusters(r"cluster/cluster-300.tsv")

print('regiao CRF binario, cluster 300, janela 4')

OUTPUT_PATH = "CRF"
OUTPUT_FILE = "crf_model_binario_regiao"
crf = joblib.load(os.path.join(OUTPUT_PATH, OUTPUT_FILE))
y_pred = crf.predict(X_test)
print(flat_classification_report(
    y_test, y_pred, labels=getTiposEntidade(), digits=3
))

regiao CRF binario, cluster 300, janela 4
              precision    recall  f1-score   support

           O      0.948     0.944     0.946      2730
         ENT      0.848     0.859     0.853       991

    accuracy                          0.921      3721
   macro avg      0.898     0.901     0.900      3721
weighted avg      0.922     0.921     0.921      3721



In [116]:
CLUSTER=300
word2cluster = read_clusters(r"cluster/cluster-300.tsv")

print('agora, com pipeline... regiao CRF binario, cluster 300, janela 4')

listacombinacoesTestPred = getListaCombinacoes(dic_predictions)

X_test = [sent2features(s, word2cluster, dicPostagger) for s in listacombinacoesTestPred]
y_test = [sent2labels(s) for s in listacombinacoesTest]

OUTPUT_PATH = "CRF"
OUTPUT_FILE = "crf_model_binario_regiao"
crf = joblib.load(os.path.join(OUTPUT_PATH, OUTPUT_FILE))
y_pred = crf.predict(X_test)
print(flat_classification_report(
    y_test, y_pred, labels=getTiposEntidade(), digits=3
))

agora, com pipeline... regiao CRF binario, cluster 300, janela 4
              precision    recall  f1-score   support

           O      0.809     0.907     0.855      1307
         ENT      0.896     0.790     0.840      1334

    accuracy                          0.848      2641
   macro avg      0.853     0.848     0.847      2641
weighted avg      0.853     0.848     0.847      2641



In [297]:
# com CRF ner - BERT

CLUSTER=300
word2cluster = read_clusters(r"cluster/cluster-300.tsv")

print('agora, com pipeline... regiao CRF binario com NER BERT, cluster 300, janela 4')

listacombinacoesTestPred, dicPred = getListaCombinacoes(dic_predictions)


X_test = [sent2features(s, word2cluster, dicPostagger) for s in listacombinacoesTestPred]
#y_test = [sent2labels(s) for s in listacombinacoesTest]

OUTPUT_PATH = "CRF"
OUTPUT_FILE = "crf_model_binario_regiao"
crf = joblib.load(os.path.join(OUTPUT_PATH, OUTPUT_FILE))
y_pred = crf.predict(X_test)



agora, com pipeline... regiao CRF binario com NER BERT, cluster 300, janela 4


In [298]:
dicPred

{0: 1,
 1: 2,
 2: 3,
 3: 4,
 4: 6,
 5: 7,
 6: 8,
 7: 9,
 8: 10,
 9: 11,
 10: 12,
 11: 13,
 12: 14,
 13: 15,
 14: 16,
 15: 17,
 16: 18,
 17: 19,
 18: 20,
 19: 23,
 20: 24,
 21: 25,
 22: 26,
 23: 27,
 24: 28,
 25: 30,
 26: 31,
 27: 32,
 28: 33,
 29: 34,
 30: 35,
 31: 36,
 32: 37,
 33: 38,
 34: 41,
 35: 42,
 36: 43,
 37: 44,
 38: 45,
 39: 46,
 40: 47,
 41: 48,
 42: 49,
 43: 50,
 44: 51,
 45: 52,
 46: 53,
 47: 54,
 48: 56,
 49: 58,
 50: 59,
 51: 60,
 52: 61,
 53: 62,
 54: 63,
 55: 64,
 56: 65,
 57: 66,
 58: 67,
 59: 68,
 60: 69,
 61: 70,
 62: 71,
 63: 72,
 64: 74,
 65: 75,
 66: 76,
 67: 77,
 68: 78,
 69: 79,
 70: 80,
 71: 82,
 72: 83,
 73: 84,
 74: 85,
 75: 86,
 76: 87,
 77: 88,
 78: 89,
 79: 90,
 80: 93,
 81: 94,
 82: 95,
 83: 97,
 84: 98,
 85: 99,
 86: 101,
 87: 103,
 88: 104,
 89: 105,
 90: 106,
 91: 107,
 92: 109,
 93: 110,
 94: 111,
 95: 112,
 96: 113,
 97: 114,
 98: 115,
 99: 116,
 100: 117,
 101: 118,
 102: 119,
 103: 121,
 104: 123,
 105: 124,
 106: 130,
 107: 131,
 108: 132,
 109:

In [305]:
print(len(dic_sentencesTest))
print(dic_sentencesTest[503])

506
[[['EX', 0, 1151], ['DE', 1, 1154], ['CONTROLE', 2, 1157], ['.', 3, 1165]], [['EX', [0], 'Teste']]]


In [212]:
dic_predictions2={}
num=0
for key, value in dic_predictions.items():
    if value[1]:
        dic_predictions2[num] = dic_predictions[key]
        num=num+1
        
        #dic_sentencesTest
    

In [246]:
dic_sentencesTest2={}
num=0
for key, value in dic_sentencesTest.items():
    if value[1]:
        dic_sentencesTest2[num] = dic_sentencesTest[key]
        num=num+1
        
        #dic_sentencesTest
    

In [273]:
dic_predictions2[64]

[[['ARea', 0], ['de', 1], ['1', 2], [',', 3], ['12', 4], ['.', 5]],
 [['ARea', [0], 'Teste']]]

In [287]:
dic_sentencesTest[5]

[[['S', 0, 448], [':', 1, 450], ['Assintomática', 2, 452], ['.', 3, 465]], []]

In [307]:
regiao_true=list()
regiao_pred=list()

dicLabels={'O':'0', 'ENT':'1', 'Problema':'1', 'Tratamento':'1', 'Teste':'1', 'Anatomia':'1'}

num=0
for ent, label in zip(listacombinacoesTestPred, y_pred):
    #print(num)
    #print(ent)
    numKey=dicPred[num]
    listaTrue=dic_sentencesTest[numKey][1]
    num=num+1
    #print(listaTrue)
    for entidade in ent:
        textoEntidade=entidade[0]
        labelEntidade=dicLabels[entidade[3]]
        labelEntidadeTrue='0'
        #print(textoEntidade)
        tem=0
        for entTrue in listaTrue:
            textoEntidadeTrue=entTrue[0]
            if textoEntidade == textoEntidadeTrue:
                #print('temmm:', textoEntidade)
                labelEntidadeTrue = dicLabels[entTrue[2]]
                tem=1
                break
        if tem==1:
            if labelEntidade == labelEntidadeTrue:
                regiao_true.append(labelEntidadeTrue)
                regiao_pred.append(labelEntidade)
            else:
                #print('aaaaaaaaaaaaa',textoEntidade)
                #print(num)
                regiao_true.append(labelEntidadeTrue)
                regiao_pred.append(labelEntidade)
        else:
            #if labelEntidade!='0':
                #print('nao temmm:{}, {}'.format(textoEntidade,labelEntidade))
                #print(num)
            regiao_true.append(labelEntidadeTrue)
            regiao_pred.append(labelEntidade)
                
    #if num>40:
    #    break
print('len(regiao_pred):', len(regiao_pred))
print('len(regiao_true):', len(regiao_true))

print('regiao_pred[:4]:', regiao_pred[:4])
print('regiao_true[:4]:', regiao_true[:4])

print(flat_classification_report(
    regiao_pred, regiao_true, labels=['1','0'], digits=3
))

len(regiao_pred): 3355
len(regiao_true): 3355
regiao_pred[:4]: ['1', '1', '0', '0']
regiao_true[:4]: ['1', '1', '0', '0']
              precision    recall  f1-score   support

           1      0.841     0.849     0.845       906
           0      0.944     0.941     0.942      2449

    accuracy                          0.916      3355
   macro avg      0.893     0.895     0.894      3355
weighted avg      0.916     0.916     0.916      3355



In [166]:
getTiposEntidade()

['O', 'ENT']

## Agora, ner CRF 

In [317]:
# com CRF ner - BERT

#transformar y_pred em dic_predictions
dic_predictions = f.getDicPredictions(y_pred, allTokens)


CLUSTER=300
word2cluster = read_clusters(r"cluster/cluster-300.tsv")

print('agora, com pipeline... regiao CRF binario com NER BERT, cluster 300, janela 4')

listacombinacoesTestPred, dicPred = getListaCombinacoes(dic_predictions)


X_test = [sent2features(s, word2cluster, dicPostagger) for s in listacombinacoesTestPred]
#y_test = [sent2labels(s) for s in listacombinacoesTest]

OUTPUT_PATH = "CRF"
OUTPUT_FILE = "crf_model_binario_regiao"
crf = joblib.load(os.path.join(OUTPUT_PATH, OUTPUT_FILE))
y_pred = crf.predict(X_test)

regiao_true=list()
regiao_pred=list()

dicLabels={'O':'0', 'ENT':'1', 'Problema':'1', 'Tratamento':'1', 'Teste':'1', 'Anatomia':'1'}

num=0
for ent, label in zip(listacombinacoesTestPred, y_pred):
    #print(num)
    #print(ent)
    numKey=dicPred[num]
    listaTrue=dic_sentencesTest[numKey][1]
    num=num+1
    #print(listaTrue)
    for entidade in ent:
        textoEntidade=entidade[0]
        labelEntidade=dicLabels[entidade[3]]
        labelEntidadeTrue='0'
        #print(textoEntidade)
        tem=0
        for entTrue in listaTrue:
            textoEntidadeTrue=entTrue[0]
            if textoEntidade == textoEntidadeTrue:
                #print('temmm:', textoEntidade)
                labelEntidadeTrue = dicLabels[entTrue[2]]
                tem=1
                break
        if tem==1:
            if labelEntidade == labelEntidadeTrue:
                regiao_true.append(labelEntidadeTrue)
                regiao_pred.append(labelEntidade)
            else:
                #print('aaaaaaaaaaaaa',textoEntidade)
                #print(num)
                regiao_true.append(labelEntidadeTrue)
                regiao_pred.append(labelEntidade)
        else:
            #if labelEntidade!='0':
                #print('nao temmm:{}, {}'.format(textoEntidade,labelEntidade))
                #print(num)
            regiao_true.append(labelEntidadeTrue)
            regiao_pred.append(labelEntidade)
                
    #if num>40:
    #    break
print('len(regiao_pred):', len(regiao_pred))
print('len(regiao_true):', len(regiao_true))

print('regiao_pred[:4]:', regiao_pred[:4])
print('regiao_true[:4]:', regiao_true[:4])

print(flat_classification_report(
    regiao_pred, regiao_true, labels=['1','0'], digits=3
))

agora, com pipeline... regiao CRF binario com NER BERT, cluster 300, janela 4
len(regiao_pred): 3456
len(regiao_true): 3456
regiao_pred[:4]: ['1', '1', '0', '0']
regiao_true[:4]: ['1', '1', '0', '0']
              precision    recall  f1-score   support

           1      0.796     0.821     0.808       871
           0      0.939     0.929     0.934      2585

    accuracy                          0.902      3456
   macro avg      0.868     0.875     0.871      3456
weighted avg      0.903     0.902     0.902      3456



In [318]:
listacombinacoesTestPred

[[['FA', ['há', '5', 'anos', 'por'], [',', 'uso', 'de', 'marevan'], 'ENT'],
  ['marevan 5mg', ['FA', ',', 'uso', 'de'], ['1', 'x', 'ao', 'dia'], 'ENT'],
  ['marevan', ['FA', ',', 'uso', 'de'], ['5mg', '1', 'x', 'ao'], 'O'],
  ['5mg', [',', 'uso', 'de', 'marevan'], ['1', 'x', 'ao', 'dia'], 'O']],
 [['Comorbidades', [], [':', 'DM', 'há', '10'], 'ENT'],
  ['DM', ['Comorbidades', ':'], ['há', '10', 'anos', 'em'], 'ENT'],
  ['metformina 850mg',
   ['anos', 'em', 'uso', 'de'],
   ['3', 'cp', '/', 'dia'],
   'ENT'],
  ['metformina', ['anos', 'em', 'uso', 'de'], ['850mg', '3', 'cp', '/'], 'O'],
  ['850mg', ['em', 'uso', 'de', 'metformina'], ['3', 'cp', '/', 'dia'], 'O'],
  ['acarbose', ['cp', '/', 'dia', ','], ['1', 'cp', '/', 'dia'], 'ENT'],
  ['glicazida 60mg', ['cp', '/', 'dia', 'e'], ['2', 'cp', '/', 'dia'], 'ENT'],
  ['glicazida', ['cp', '/', 'dia', 'e'], ['60mg', '2', 'cp', '/'], 'O'],
  ['60mg', ['/', 'dia', 'e', 'glicazida'], ['2', 'cp', '/', 'dia'], 'O'],
  ['insulina', ['cp', '/', 'd

# Agora,  pipeline completo

In [319]:
from simpletransformers.classification import (
    ClassificationModel, ClassificationArgs
)
import pandas as pd
import logging

print('-----Avaliando só com modelo de Sentence Pairs:-----')
model = ClassificationModel('bert', 'lisaterumi/sentence_pairs_nested_positivos', use_cuda=False)


-----Avaliando só com modelo de Sentence Pairs:-----


In [328]:
def getCombinacaoEntidadesSentence(dic_predictions, filtro_postagger, dicPosTagger, taxaDownsampling, lista_postaggers_entidades):
    #labels = {0:'O', 1:'Problema', 2:'Tratamento', 3:'Teste', 4:'Anatomia'}
    #labels = {'O':0, 'Problema':1, 'Tratamento':2, 'Teste':3, 'Anatomia':4}
    labels = {'O':0, 'ENT':1}
    num=0
    erro_corpus=0
    num_frases_sem_entidade=0
    lista_erro_corpus=list()
    combinacaoEntidadesPos = list()
    combinacaoEntidadesNeg = list()
    combinacaoEntidades = list()
    combinacaoEntidadesAll = list()
    pulando_termos_postagger = list()
    if filtro_postagger:
        print('Sentence Pairs - Com filtro-postagger')
    else:
        print('Sentence Pairs - Sem filtro-postagger')
    if taxaDownsampling>0:
        print('Sentence Pairs - Com taxa de Downsampling de ', taxaDownsampling)
    else:
        print('Sentence Pairs - Sem taxa de Downsampling')

    for key, value in dic_predictions.items():
        num=num+1
        tokens=value[0].copy()
        so_tokens = [t[0] for t in tokens]
        entidades=value[1].copy()
        num_positivas=0
        lista_indices = list()
        for entidade in entidades:
            erros_entidade = list()
            texto_entidade=entidade[0].strip()
            indices = entidade[1]
            tipo_entidade = entidade[2]
            frase = so_tokens.copy()
            inicio=indices[0]
            fim=indices[-1]
            lista_indices = [inicio, fim]
            #entidade_frase=frase[inicio:fim+1] # texto_entidade
            entidade_frase=texto_entidade
            #print('entidade_frase:', entidade_frase)
            #print('frase:', frase)
            #print('texto_entidade:', texto_entidade)
            if texto_entidade=='-' or texto_entidade=='=' or texto_entidade=='+' or texto_entidade==':' or texto_entidade==',' or texto_entidade=="'" or texto_entidade=='"' or texto_entidade=='.' or texto_entidade==';' or texto_entidade=='/' or texto_entidade=='(' or texto_entidade==')' or texto_entidade=='[' or texto_entidade==']':
                pass
            texto_entidade_comparar=texto_entidade.replace('/','').replace(')','').replace('(','').replace(']','').replace('[','').replace(',','').replace('.','').replace(';','').replace('-','').replace('+','').replace("'",'')
            texto_entidade_comparar = f.replaceWhiteSpaces(texto_entidade_comparar)
            texto_frase_comparar = ' '.join(frase[inicio:fim+1]).strip().replace('/','').replace(')','').replace('(','').replace(']','').replace('[','').replace(',','').replace('.','').replace(';','').replace('-','').replace('+','').replace("'",'')
            texto_frase_comparar = f.replaceWhiteSpaces(texto_frase_comparar)
            texto_entidade_comparar = texto_entidade_comparar.lower()
            texto_frase_comparar = texto_frase_comparar.lower()
            if (texto_entidade_comparar == texto_frase_comparar):
                num_positivas=num_positivas+1
                lista_indices_proc = [num for num in range(lista_indices[0], lista_indices[1]+1, 1)]
                combinacaoEntidadesPos.append([entidade_frase, ' '.join(frase).strip(), lista_indices_proc, labels[tipo_entidade]]) # apendando entidades reais
            else:
                print('erro, key:', key)
                erro_corpus=erro_corpus+1
                erros_entidade.append(indices)
                lista_erro_corpus.append([' '.join(frase).strip(), tipo_entidade, ' '.join(so_tokens), entidade])
        # agora, os negativos
        for entidade in entidades:
                indices = entidade[1]
                #print('indices:', indices)
                if indices in erros_entidade:
                    continue
                inicio=indices[0]
                fim=indices[-1]
                # agora, fazer a combinacao entre eles.. todas a seguir serão do tipo 'O'           
                for indice in indices:
                    for i in range(indice, fim+1):
                        # ver se nao tem antes
                        frase = so_tokens.copy()
                        termo = frase[indice:i+1]
                        frase_string=' '.join(termo).strip()
                        devePular = 0
                        if '.' in frase_string[-1:] or ',' in frase_string[-1:]  or ';' in frase_string[-1:] or '-' in frase_string[-1:]  or ':' in frase_string[-1:]  or '=' in frase_string[-1:]  or '/' in frase_string[-1:]  or '(' in frase_string[-1:]  or ')' in frase_string[-1:]  or '[' in frase_string[-1:]  or ']' in frase_string[-1:]  or ':' in frase_string[-1:]:
                            devePular=1
                        if '.' in frase_string[:1] or ',' in frase_string[:1]  or ';' in frase_string[:1] or '-' in frase_string[:1]  or ':' in frase_string[:1]  or '=' in frase_string[:1] or '/' in frase_string[:1]  or '(' in frase_string[:1]  or ')' in frase_string[:1] or '[' in frase_string[:1]  or ']' in frase_string[:1]  or ':' in frase_string[:1]:
                            devePular=1
                        if re.search("^[0-9]*mg", frase_string):
                            devePular=1
                            
                        if filtro_postagger==True:
                            pos_tagger_termo = f.tipoPostaggerTokens(termo, dicPosTagger)
                            if pos_tagger_termo not in lista_postaggers_entidades:
                                pulando_termos_postagger.append([termo, pos_tagger_termo])
                                devePular=1
                
                        tem_frase = 0
                        for frase_l in combinacaoEntidadesPos:
                            if frase_l[0] == frase_string:
                                tem_frase='1'
                                break
                        if tem_frase==0 and devePular==0:
                        #print('tem_frase:', tem_frase)
                        #if tem_frase==0:
                            lista_indices = [indice, len(frase_string.split())-1+indice]
                            lista_indices_proc = [num for num in range(lista_indices[0], lista_indices[1]+1, 1)]
                            combinacaoEntidadesNeg.append([frase_string, ' '.join(frase).strip(), lista_indices_proc, labels['O']])
                        
        # shuffle no combinacaoEntidades
        # taxaDownsampling, ex 2 para o dobro, 1 para mesma quantidade
        if len(combinacaoEntidadesPos)>0:
            if taxaDownsampling>0:
                combinacaoEntidadesNeg = combinacaoEntidadesNeg[:(num_positivas*taxaDownsampling)+num_positivas]
            random.shuffle(combinacaoEntidadesNeg)
        else:
            num_frases_sem_entidade = num_frases_sem_entidade+1
        if (num % 1000) ==0:
            print('key:', key)

        #print('combinacaoEntidadesNeg:',combinacaoEntidadesNeg)
        #combinacaoEntidades = combinacaoEntidades+combinacaoEntidadesPos+combinacaoEntidadesNeg
        combinacaoEntidades = combinacaoEntidadesPos+combinacaoEntidadesNeg
        combinacaoEntidadesPos=list()
        combinacaoEntidadesNeg=list()
        combinacaoEntidadesAll.append(combinacaoEntidades)
  
    print('erro_corpus:', erro_corpus)
    print('num_frases_sem_entidade:', num_frases_sem_entidade)
    print('len(combinacaoEntidadesAll:)',len(combinacaoEntidadesAll))
    
    return combinacaoEntidadesAll

In [329]:
dic_sentencesTrainDev = f.load_obj('dic_sentencesTrainDev')
dicPosTagger, _ = f.getDicPosTagger(dic_sentencesTrainDev)
lista_postaggers_entidades = f.getListaPostaggerEntidades(dic_predictions, dicPosTagger)
# para gerar arquivo de predicoes (com as tags <e1>)
print('Primeiro, com filtro postagger:')
combinacaoEntidadesAll = getCombinacaoEntidadesSentence(dic_predictions, True, dicPosTagger, 0, lista_postaggers_entidades)

combinacaoEntidadesAll[20]

Primeiro, com filtro postagger:
Sentence Pairs - Com filtro-postagger
Sentence Pairs - Sem taxa de Downsampling
erro_corpus: 0
num_frases_sem_entidade: 97
len(combinacaoEntidadesAll:) 506


[['sinvastatina para 40mg',
  'Otimizo dose da sinvastatina para 40mg / dia .',
  [3, 4, 5],
  1],
 ['sinvastatina para',
  'Otimizo dose da sinvastatina para 40mg / dia .',
  [3, 4],
  0],
 ['para', 'Otimizo dose da sinvastatina para 40mg / dia .', [4], 0],
 ['sinvastatina', 'Otimizo dose da sinvastatina para 40mg / dia .', [3], 0]]

In [330]:
%%capture 
# para suprimir output
#combinacaoEntidadesAll = f.load_obj('combinacaoEntidadesAll_'+str(BATCH))
print('Chamando predict')
pred_region_labels = list()
for key, combinacao in enumerate(combinacaoEntidadesAll):
    if key<BATCH:    
        if len(combinacao)>0:
            lista = [l[0:2] for l in combinacao]
            predictions, _ = model.predict(lista) 
            pred_region_labels.append(predictions)
            for comb, label in zip(combinacao, predictions):
                comb.append(label)

In [332]:
def getDicPredictionsSentence(combinacaoEntidadesAll_pred, dic_predictions):
    labels = {0:'O', 1:'Problema', 2:'Tratamento', 3:'Teste', 4:'Anatomia'}
    num=-1
    dic_predictions_sentence={}
    entidades=list()
    numAcrescentou=0
    for frases in combinacaoEntidadesAll_pred:
        num=num+1
        for valor in frases:
            #print('num:', num)
            #print('valor:', valor)
            #print('valor[1]:', valor[1])
            tokens_entidade = valor[0]
            #frase = valor[1]
            indices = valor[2]
            tipo_previsto=valor[4]
            #print('tokens_entidade:', tokens_entidade)
            #print('frase:', frase)
            #print('indices:', indices)
            #print('tipo_previsto:', tipo_previsto)
            if tipo_previsto!=0:
                entidades.append([tokens_entidade, indices, labels[tipo_previsto]])
            # ver se entidade está na dic_prediction
        frase = dic_predictions[num].copy()[0]
        dic_predictions_sentence[num] = [frase, entidades]
        entidades = list()
                     
    return dic_predictions_sentence

In [337]:
f.save_obj('dic_predictions_sentence_CRF_pipeline', dic_predictions_sentence)

In [333]:
dic_predictions_sentence = getDicPredictionsSentence(combinacaoEntidadesAll, dic_predictions)
dic_predictions_sentence[20]

[[['Otimizo', 0],
  ['dose', 1],
  ['da', 2],
  ['sinvastatina', 3],
  ['para', 4],
  ['40mg', 5],
  ['/', 6],
  ['dia', 7],
  ['.', 8]],
 [['sinvastatina para 40mg', [3, 4, 5], 'Problema'],
  ['sinvastatina para', [3, 4], 'Problema'],
  ['para', [4], 'Problema'],
  ['sinvastatina', [3], 'Problema']]]

In [334]:
print('-----Avaliando modelo sem pipeline:-----')

region_true_list, region_pred_list = f.AvalFinal(dicSentences_new_test, dic_predictions_sentence, BATCH)

-----Avaliando modelo sem pipeline:-----
numErro1: 408
numErro2: 720
              precision    recall  f1-score   support

    Anatomia   0.000000  0.000000  0.000000     196.0
           O   0.000000  0.000000  0.000000     720.0
    Problema   0.000000  0.000000  0.000000     338.0
       Teste   0.000000  0.000000  0.000000     243.0
  Tratamento   0.000000  0.000000  0.000000     214.0

    accuracy                       0.000000    1711.0
   macro avg   0.000000  0.000000  0.000000    1711.0
weighted avg   0.000000  0.000000  0.000000    1711.0



C:\Users\lisat\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# ========================

In [76]:
example_sent = testdata[18]
print("\nSentence:", ' '.join(sent2tokens(example_sent)))
print(' '.join(sent2tokens(example_sent)))
print("Predicted:", ' '.join(crf.predict([sent2features(example_sent)])[0]))
print("Correct:  ", ' '.join(sent2labels(example_sent)))



Sentence: Abdomen flácido , indolor , sem visceromegalias .
Abdomen flácido , indolor , sem visceromegalias .
Predicted: Teste O O O O O Problema O
Correct:   Anatomia O O O O O Problema O


In [27]:
with open('predicts_crf_1nivel.csv','w') as arq:
    for i in range(0,len(testdata)):
        example_sent = testdata[i]
        #arq.write("\n\nSentence:"+ ' '.join(sent2tokens(example_sent)))
        #arq.write("\nPredicted:"+ ' '.join(crf.predict([sent2features(example_sent)])[0]))
        #arq.write("\nCorrect:  "+ ' '.join(sent2labels(example_sent)))
        arq.write('"')
        arq.write('","'.join(sent2tokens(example_sent)))
        arq.write('"\n"')
        arq.write('","'.join(crf.predict([sent2features(example_sent)])[0]))
        arq.write('"\n"')
        arq.write('","'.join(sent2labels(example_sent)))
        arq.write('"\n\n')
        #print("\nSentence:", ' '.join(sent2tokens(example_sent)))
        #print("Predicted:", ' '.join(crf.predict([sent2features(example_sent)])[0]))
        #print("Correct:  ", ' '.join(sent2labels(example_sent)))


In [37]:
for i in range(0,5):
    example_sent = testdata[i]
    print("\nSentence:", ' '.join(sent2tokens(example_sent)))
    print("Predicted:", ' '.join(crf.predict([sent2features(example_sent)])[0]))
    print("Correct:  ", ' '.join(sent2labels(example_sent)))



Sentence: Lucas , 74 anos .
Predicted: O O O O O
Correct:   O O O O O

Sentence: Em acompanhamento no ambualtorio há 5 anos por FA , uso de marevan 5mg 1 x ao dia .
Predicted: O O O O O O O O Problema O O O Tratamento Tratamento O O O O O
Correct:   O O O O O O O O Problema O O O Tratamento Tratamento O O O O O

Sentence: Comorbidades : DM há 10 anos em uso de metformina 850mg 3 cp / dia , acarbose 1 cp / dia e glicazida 60mg 2 cp / dia e insulina ( 24 - 0 - 24 ) .
Predicted: Problema O Problema O O O O O O Tratamento Tratamento O O O O O O O O O O O Tratamento Tratamento O O O O O Tratamento O O O O O O O O
Correct:   Problema O Problema O O O O O O Tratamento Tratamento O O O O O Tratamento O O O O O Tratamento Tratamento O O O O O Tratamento O O O O O O O O

Sentence: HAS há 15 anos em uso de losartana 50mg / dia e digoxina 1 / 2 cp / dia , carvedilol 25 12 / 12 , HCTZ .
Predicted: Problema O O O O O O Tratamento Tratamento O O O Teste O O O O O O O Tratamento Tratamento O O O O O 